In [1]:
import sys
import os
import matplotlib
%matplotlib inline
import argparse

from scripts.nbfinder import NotebookFinder
sys.meta_path.append(NotebookFinder())

In [2]:
'''before we import theano anywhere else we want to make sure we specify 
a unique directory for compiling, so we dont get into a locking issue
if we run multiple hur_mains at once on a global file system. Haven't truly implementedthis yet '''
from notebooks.run_dir import create_run_dir
from notebooks.helper_fxns import dump_hyperparams
from notebooks.data_loader import load_data, load_precomputed_data
from notebooks.train_val import train
from notebooks.print_n_plot import plot_ims_with_boxes
from notebooks.build_network import build_network

importing Jupyter notebook from notebooks/run_dir.ipynb
importing Jupyter notebook from notebooks/helper_fxns.ipynb
importing Jupyter notebook from notebooks/data_loader.ipynb
importing Jupyter notebook from notebooks/train_val.ipynb
importing Jupyter notebook from notebooks/build_network.ipynb
importing Jupyter notebook from notebooks/print_n_plot.ipynb


In [3]:
# if inside a notebook, then get rid of weird notebook arguments, so that arg parsing still works
if any(["jupyter" in arg for arg in sys.argv]):
    sys.argv=sys.argv[:1]
    

parser = argparse.ArgumentParser()
parser.add_argument('-e', '--epochs', type=int, default=10000,
    help='number of epochs for training')

parser.add_argument('-l', '--learn_rate', default=0.0001, type=float,
    help='the learning rate for the network')

parser.add_argument('-n', '--num_ims', default=30, type=int,
    help='number of total images')

parser.add_argument('-f', '--num_filters', default=128, type=int,
    help='number of filters in each conv layer')

parser.add_argument( '--fc', default=512, type=int,
    help='number of fully connected units')

parser.add_argument('--coord_penalty', default=5, type=int,
    help='penalty for guessing coordinates wrong')

parser.add_argument('--size_penalty', default=5, type=int,
    help='penalty for guessing height or width wrong')

parser.add_argument('--nonobj_penalty', default=0.5, type=float,
    help='penalty for guessing an object where one isnt')

parser.add_argument('-c','--num_extra_conv', default=0, type=int,
    help='conv layers to add on to each conv layer before max pooling')

parser.add_argument('--num_convpool', default=4, type=int,
    help='number of conv layer-pool layer pairs')

parser.add_argument('--momentum', default=0.9, type=float,
    help='momentum')


args = parser.parse_args()

In [4]:

run_dir = create_run_dir()
print run_dir

dataset = load_precomputed_data(paths=["/global/project/projectdirs/dasrepo/gordon_bell/climate/data/detection/caffe_data/chur_train.h5",
                                      "/global/project/projectdirs/dasrepo/gordon_bell/climate/data/detection/caffe_data/chur_val.h5" ],
                                out_of_core=True)
# dataset = load_data(num_ims=args.num_ims,
#                     path='/project/projectdirs/dasrepo/gordon_bell/climate/data/detection/hur_train_val.h5')

'''size of ground truth grid'''
grid_size = dataset[1].shape[1]

'''set params'''
network_kwargs = {'learning_rate': args.learn_rate, 
                  'dropout_p': 0, 
                  'weight_decay': 0, 
                  'num_filters': args.num_filters, 
                  'num_fc_units': args.fc, 
                  'num_convpool': args.num_convpool,
                  'num_extra_conv': args.num_extra_conv,
                  'momentum': args.momentum,
                  'coord_penalty': args.coord_penalty,
                  'nonobj_penalty': args.nonobj_penalty,
                   }


'''get network and train_fns'''
train_fn, val_fn, box_fn, network, hyperparams = build_network(**network_kwargs)

hyperparams.update({'num_ims': args.num_ims, 'tr_size': dataset[0].shape[0]})
'''save hyperparams'''
dump_hyperparams(hyperparams, path=run_dir)

'''train'''
train(dataset, network=network, fns=(train_fn, val_fn, box_fn),num_ims=args.num_ims, save_weights=True, num_epochs=args.epochs, save_path=run_dir)

/global/u1/r/racah/projects/mantissa-climate/results/run101
Building model and compiling functions...
Starting training...


Epoch 1 of 10000 took 5.821s
	training los:		11.9792
	training acc:		0.0237 %
  validation loss:		12.390483
  validation accuracy:		0.22 %


KeyboardInterrupt: 